In [163]:
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import os
import random
# 한글 문제
import platform

from matplotlib import font_manager, rc
plt.rcParams['axes.unicode_minus'] = False

if platform.system() == 'Darwin':  # 맥OS
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':  # 윈도우
    path = "c:/Windows/Fonts/malgun.ttf"
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print('Unknown system...  sorry~~~')

from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 17,8

import warnings
warnings.filterwarnings(action='ignore')

# **대구 교통 사고 피해 예측 AI 경진대회 Baseline Code**

## **Fixed Random Seed**  

seed 값에 의해 동일한 코드를 사용해도 결과가 다를 수 있기에, 동일한 결과를 위해 seed 값을 고정시킵니다

In [164]:

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42)

## **데이터 불러오기 및 상위행 확인**  

train.csv, test.csv 파일을 로드하여 상위행을 출력해 봅시다

In [165]:

train = pd.read_csv('./open/train.csv', encoding = "utf-8", low_memory=False)
test = pd.read_csv('./open/test.csv', encoding = "utf-8", low_memory=False)
countrywide = pd.read_csv('./open/external_open/countrywide_accident.csv')


## **train, test 데이터 기간 확인하기**  

학습(train) 데이터의 기간과 예측 대상이 되는 test 데이터의 기간을 살펴 봅니다

In [166]:
#'사고일시' 데이터 타입을 object -> datetime으로  바꾸기
train['사고일시'] = pd.to_datetime(train['사고일시'])
test['사고일시'] = pd.to_datetime(test['사고일시'])
countrywide['사고일시'] = pd.to_datetime(countrywide['사고일시'])

### - train과 test 데이터셋의  컬럼 수가 서로 다름
    - ID, 사고일시, 요일, 기상상태, 시군구, 도로형태, 노면상태, 사고유형

In [167]:
display(train['요일'].unique())
display(test['요일'].unique())
display(countrywide['요일'].unique())

array(['화요일', '수요일', '목요일', '금요일', '토요일', '일요일', '월요일'], dtype=object)

array(['토요일', '일요일', '월요일', '화요일', '수요일', '목요일', '금요일'], dtype=object)

array(['화요일', '수요일', '목요일', '금요일', '토요일', '일요일', '월요일'], dtype=object)

In [168]:
display(train['기상상태'].unique())
display(test['기상상태'].unique())
display(countrywide['기상상태'].unique())

array(['맑음', '흐림', '기타', '비', '안개', '눈'], dtype=object)

array(['맑음', '흐림', '기타', '비', '눈'], dtype=object)

array(['맑음', '흐림', '눈', '기타', '비', '안개'], dtype=object)

***- test에는 기상상태-'안개' 없음***

## **데이터 전처리**  

현재 '사고일시', '시군구', '도로형태' 컬럼은 반복되는 패턴으로 여러 정보를 포함하고 있습니다
이런 반복되는 패턴을 일반화하면 pandas에서 제공하는 str.extract를 통해 한 번에 추출 가능합니다  

## **파생 변수 생성 1 : 날짜, 시간정보 생성**

'사고일시' 컬럼으로 부터 연도, 월, 일, 시간 정보 추출 및 변환 합니다

In [169]:
print(test['사고일시'].dtype)

datetime64[ns]


In [170]:
train_df = train.copy()
test_df = test.copy()
countrywide_df = countrywide.copy()

time_pattern = r'(\d{4})-(\d{1,2})-(\d{1,2}) (\d{1,2})'

train['사고일시'] = train['사고일시'].astype(str)
train_df[['연', '월', '일', '시간']] = train['사고일시'].str.extract(time_pattern)
train_df[['연', '월', '일', '시간']] = train_df[['연', '월', '일', '시간']].apply(pd.to_numeric) # 추출된 문자열을 수치화해줍니다
train_df = train_df.drop(columns=['사고일시']) # 정보 추출이 완료된 '사고일시' 컬럼은 제거합니다

# 해당 과정을 test_x에 대해서도 반복해줍니다
test['사고일시'] = test['사고일시'].astype(str)
test_df[['연', '월', '일', '시간']] = test['사고일시'].str.extract(time_pattern)
test_df[['연', '월', '일', '시간']] = test_df[['연', '월', '일', '시간']].apply(pd.to_numeric)
test_df = test_df.drop(columns=['사고일시'])

countrywide['사고일시'] = countrywide['사고일시'].astype(str)
countrywide_df[['연', '월', '일', '시간']] = countrywide['사고일시'].str.extract(time_pattern)
countrywide_df[['연', '월', '일', '시간']] = countrywide_df[['연', '월', '일', '시간']].apply(pd.to_numeric)
countrywide_df = countrywide_df.drop(columns=['사고일시'])



display(f"columns of train_df : {train_df.columns}")
display(f"columns of test_df : {test_df.columns}")
display(f"columns of test_df : {countrywide_df.columns}")

"columns of train_df : Index(['ID', '요일', '기상상태', '시군구', '도로형태', '노면상태', '사고유형', '사고유형 - 세부분류',\n       '법규위반', '가해운전자 차종', '가해운전자 성별', '가해운전자 연령', '가해운전자 상해정도', '피해운전자 차종',\n       '피해운전자 성별', '피해운전자 연령', '피해운전자 상해정도', '사망자수', '중상자수', '경상자수', '부상자수',\n       'ECLO', '연', '월', '일', '시간'],\n      dtype='object')"

"columns of test_df : Index(['ID', '요일', '기상상태', '시군구', '도로형태', '노면상태', '사고유형', '연', '월', '일', '시간'], dtype='object')"

"columns of test_df : Index(['ID', '요일', '기상상태', '시군구', '도로형태', '노면상태', '사고유형', '사고유형 - 세부분류',\n       '법규위반', '가해운전자 차종', '가해운전자 성별', '가해운전자 연령', '가해운전자 상해정도', '피해운전자 차종',\n       '피해운전자 성별', '피해운전자 연령', '피해운전자 상해정도', '사망자수', '중상자수', '경상자수', '부상자수',\n       'ECLO', '연', '월', '일', '시간'],\n      dtype='object')"

## **파생 변수 생성 2 : 공간(위치) 정보 생성**

'시군구' 컬럼으로부터 의미 있는 공산 정보를 추출 및 변환 합니다

In [171]:
location_pattern = r'(\S+) (\S+) (\S+)'

train_df[['도시', '구', '동']] = train['시군구'].str.extract(location_pattern)
train_df = train_df.drop(columns=['시군구'])

test_df[['도시', '구', '동']] = test['시군구'].str.extract(location_pattern)
test_df = test_df.drop(columns=['시군구'])

countrywide_df[['도시', '구', '동']] = countrywide['시군구'].str.extract(location_pattern)
countrywide_df = countrywide_df.drop(columns=['시군구'])


display(f"columns of train_df : {train_df.columns}")
display(f"columns of test_df : {test_df.columns}")
display(f"columns of test_df : {countrywide_df.columns}")

"columns of train_df : Index(['ID', '요일', '기상상태', '도로형태', '노면상태', '사고유형', '사고유형 - 세부분류', '법규위반',\n       '가해운전자 차종', '가해운전자 성별', '가해운전자 연령', '가해운전자 상해정도', '피해운전자 차종',\n       '피해운전자 성별', '피해운전자 연령', '피해운전자 상해정도', '사망자수', '중상자수', '경상자수', '부상자수',\n       'ECLO', '연', '월', '일', '시간', '도시', '구', '동'],\n      dtype='object')"

"columns of test_df : Index(['ID', '요일', '기상상태', '도로형태', '노면상태', '사고유형', '연', '월', '일', '시간', '도시',\n       '구', '동'],\n      dtype='object')"

"columns of test_df : Index(['ID', '요일', '기상상태', '도로형태', '노면상태', '사고유형', '사고유형 - 세부분류', '법규위반',\n       '가해운전자 차종', '가해운전자 성별', '가해운전자 연령', '가해운전자 상해정도', '피해운전자 차종',\n       '피해운전자 성별', '피해운전자 연령', '피해운전자 상해정도', '사망자수', '중상자수', '경상자수', '부상자수',\n       'ECLO', '연', '월', '일', '시간', '도시', '구', '동'],\n      dtype='object')"

## **파생 변수 추출 3 : 도로 형태 정보 추출**  

'도로형태' 컬럼은 '단일로 - 기타'와 같은 패턴으로 구성되어 있습니다. 이를 두종류의 독립된 정보로 보고 두개의 컬럼으로 분리하여 생성합니다.

In [172]:
road_pattern = r'(.+) - (.+)'

train_df[['도로형태1', '도로형태2']] = train['도로형태'].str.extract(road_pattern)
train_df = train_df.drop(columns=['도로형태'])


test_df[['도로형태1', '도로형태2']] = test['도로형태'].str.extract(road_pattern)
test_df = test_df.drop(columns=['도로형태'])

countrywide_df[['도로형태1', '도로형태2']] = countrywide['도로형태'].str.extract(road_pattern)
countrywide_df = countrywide_df.drop(columns=['도로형태'])

display(f"columns of train_df : {train_df.columns}")
display(f"columns of test_df : {test_df.columns}")
display(f"columns of test_df : {countrywide_df.columns}")

"columns of train_df : Index(['ID', '요일', '기상상태', '노면상태', '사고유형', '사고유형 - 세부분류', '법규위반', '가해운전자 차종',\n       '가해운전자 성별', '가해운전자 연령', '가해운전자 상해정도', '피해운전자 차종', '피해운전자 성별',\n       '피해운전자 연령', '피해운전자 상해정도', '사망자수', '중상자수', '경상자수', '부상자수', 'ECLO', '연',\n       '월', '일', '시간', '도시', '구', '동', '도로형태1', '도로형태2'],\n      dtype='object')"

"columns of test_df : Index(['ID', '요일', '기상상태', '노면상태', '사고유형', '연', '월', '일', '시간', '도시', '구', '동',\n       '도로형태1', '도로형태2'],\n      dtype='object')"

"columns of test_df : Index(['ID', '요일', '기상상태', '노면상태', '사고유형', '사고유형 - 세부분류', '법규위반', '가해운전자 차종',\n       '가해운전자 성별', '가해운전자 연령', '가해운전자 상해정도', '피해운전자 차종', '피해운전자 성별',\n       '피해운전자 연령', '피해운전자 상해정도', '사망자수', '중상자수', '경상자수', '부상자수', 'ECLO', '연',\n       '월', '일', '시간', '도시', '구', '동', '도로형태1', '도로형태2'],\n      dtype='object')"

## train, test 컬럼 수 맞추기

test에 있는 컬럼들만 사용하여 train 데이터를 구성했습니다.

In [174]:
train_df= train_df[['요일','기상상태','노면상태', '사고유형', '연', '월', '시간', '도시','도로형태1', '도로형태2']]
train_df.info()
test_df = test_df[['요일','기상상태','노면상태', '사고유형', '연', '월', '시간', '도시','도로형태1', '도로형태2']]
test_df.info()
countrywide_df = countrywide_df[['요일','기상상태','노면상태', '사고유형', '연', '월', '시간', '도시','도로형태1', '도로형태2']]
countrywide_df.info()

train_df.to_csv('train_processed.csv', index=False)
test_df.to_csv('test_processed.csv', index=False)
countrywide_df.to_csv('countrywide_processed.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39609 entries, 0 to 39608
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   요일      39609 non-null  object
 1   기상상태    39609 non-null  object
 2   노면상태    39609 non-null  object
 3   사고유형    39609 non-null  object
 4   연       39609 non-null  int64 
 5   월       39609 non-null  int64 
 6   시간      39609 non-null  int64 
 7   도시      39609 non-null  object
 8   도로형태1   39609 non-null  object
 9   도로형태2   39609 non-null  object
dtypes: int64(3), object(7)
memory usage: 3.0+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10963 entries, 0 to 10962
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   요일      10963 non-null  object
 1   기상상태    10963 non-null  object
 2   노면상태    10963 non-null  object
 3   사고유형    10963 non-null  object
 4   연       10963 non-null  int64 
 5   월       10963 non-null  int64 
 6   시간     

## **전처리 결과 확인**

지금까지 전처리한 결과를 확인해 봅시다

## 데이터 초기화 (data_label_enco, data_onehot_enco)

In [67]:
# 전처리를 위해 train과 test 데이터를 data라는 데이터프레임으로 합치기
data = pd.concat([train_df, test_df], sort=False)
data_label_enco = data.copy()
data_onehot_enco = data.copy()


In [71]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50572 entries, 0 to 10962
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      50572 non-null  object
 1   요일      50572 non-null  object
 2   기상상태    50572 non-null  object
 3   노면상태    50572 non-null  object
 4   사고유형    50572 non-null  object
 5   연       50572 non-null  int64 
 6   월       50572 non-null  int64 
 7   일       50572 non-null  int64 
 8   시간      50572 non-null  int64 
 9   도시      50572 non-null  object
 10  구       50572 non-null  object
 11  동       50572 non-null  object
 12  도로형태1   50572 non-null  object
 13  도로형태2   50572 non-null  object
dtypes: int64(4), object(10)
memory usage: 5.8+ MB


# **1. 노면상태, 사고유형 엔코딩**

## Label encoding

In [22]:
from sklearn.preprocessing import LabelEncoder
# LabelEncoder 객체 생성
label_encoder = LabelEncoder()

# "사고유형" 컬럼 Label Encoding
data_label_enco['사고유형'] = label_encoder.fit_transform(data_label_enco['사고유형'])

# "노면상태" 컬럼 Label Encoding
data_label_enco['노면상태'] = label_encoder.fit_transform(data_label_enco['노면상태'])

# 결과 확인
print(data_label_enco[['사고유형', '노면상태']])
data_label_enco.info()

       사고유형  노면상태
0         0     0
1         0     0
2         0     0
3         1     0
4         1     0
...     ...   ...
10958     1     0
10959     1     0
10960     1     0
10961     1     0
10962     1     0

[50572 rows x 2 columns]
<class 'pandas.core.frame.DataFrame'>
Index: 50572 entries, 0 to 10962
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      50572 non-null  object
 1   요일      50572 non-null  object
 2   기상상태    50572 non-null  object
 3   노면상태    50572 non-null  int32 
 4   사고유형    50572 non-null  int32 
 5   연       50572 non-null  int64 
 6   월       50572 non-null  int64 
 7   일       50572 non-null  int64 
 8   시간      50572 non-null  int64 
 9   도시      50572 non-null  object
 10  구       50572 non-null  object
 11  동       50572 non-null  object
 12  도로형태1   50572 non-null  object
 13  도로형태2   50572 non-null  object
dtypes: int32(2), int64(4), object(8)
memory usage: 5.4+ MB


## One-Hot Encoding

In [23]:
# "사고유형" 및 "노면상태" 컬럼을 원-핫 인코딩
data_onehot_enco = pd.get_dummies(data, columns=['사고유형', '노면상태'])

## 전처리한 데이터로 선형회귀 및 rmsle확인

# **------------------------------------------------------**

# **2. 도로유형 인코딩**

### 라벨인코딩 - 도로유형1

In [24]:
le = LabelEncoder()
le.fit(data_label_enco['도로형태1'])

LabelEncoder()

In [25]:
le.transform(data_label_enco['도로형태1'])

array([2, 2, 2, ..., 2, 0, 0])

In [26]:
label_data = data.copy()

In [27]:
data_label_enco['도로형태1'] = le.transform(data_label_enco['도로형태1'])

In [28]:
data_label_enco.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50572 entries, 0 to 10962
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      50572 non-null  object
 1   요일      50572 non-null  object
 2   기상상태    50572 non-null  object
 3   노면상태    50572 non-null  int32 
 4   사고유형    50572 non-null  int32 
 5   연       50572 non-null  int64 
 6   월       50572 non-null  int64 
 7   일       50572 non-null  int64 
 8   시간      50572 non-null  int64 
 9   도시      50572 non-null  object
 10  구       50572 non-null  object
 11  동       50572 non-null  object
 12  도로형태1   50572 non-null  int32 
 13  도로형태2   50572 non-null  object
dtypes: int32(3), int64(4), object(7)
memory usage: 5.2+ MB


### 라벨인코딩 - 도로유형2

In [29]:
le = LabelEncoder()
le.fit(data_label_enco['도로형태2'])
le.transform(data_label_enco['도로형태2'])

data_label_enco['도로형태2'] = le.transform(data_label_enco['도로형태2'])

data_label_enco.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50572 entries, 0 to 10962
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      50572 non-null  object
 1   요일      50572 non-null  object
 2   기상상태    50572 non-null  object
 3   노면상태    50572 non-null  int32 
 4   사고유형    50572 non-null  int32 
 5   연       50572 non-null  int64 
 6   월       50572 non-null  int64 
 7   일       50572 non-null  int64 
 8   시간      50572 non-null  int64 
 9   도시      50572 non-null  object
 10  구       50572 non-null  object
 11  동       50572 non-null  object
 12  도로형태1   50572 non-null  int32 
 13  도로형태2   50572 non-null  int32 
dtypes: int32(4), int64(4), object(6)
memory usage: 5.0+ MB


### 원핫인코딩 - 도로형태1

In [30]:
from sklearn.preprocessing import OneHotEncoder

data_onehot_enco = pd.get_dummies(data_onehot_enco, columns=['도로형태1', '도로형태2'])

In [31]:
data_onehot_enco.info()
# oe_road1 = pd.DataFrame(oe_road1)

<class 'pandas.core.frame.DataFrame'>
Index: 50572 entries, 0 to 10962
Data columns (total 34 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ID               50572 non-null  object
 1   요일               50572 non-null  object
 2   기상상태             50572 non-null  object
 3   연                50572 non-null  int64 
 4   월                50572 non-null  int64 
 5   일                50572 non-null  int64 
 6   시간               50572 non-null  int64 
 7   도시               50572 non-null  object
 8   구                50572 non-null  object
 9   동                50572 non-null  object
 10  사고유형_차대사람        50572 non-null  bool  
 11  사고유형_차대차         50572 non-null  bool  
 12  사고유형_차량단독        50572 non-null  bool  
 13  노면상태_건조          50572 non-null  bool  
 14  노면상태_기타          50572 non-null  bool  
 15  노면상태_서리/결빙       50572 non-null  bool  
 16  노면상태_적설          50572 non-null  bool  
 17  노면상태_젖음/습기       50572 non-null  boo

In [32]:

# data_onehot_enco = pd.merge(data, oe_road1, left_index=True, right_index=True)

# data_onehot_enco = data_onehot_enco.drop(columns='도로형태1')
# data_onehot_enco = data_onehot_enco.rename(columns= {0 : '도로형태1'})

# data_onehot_enco.info()

### 원핫인코딩 - 도로형태2 -> (수정) : 위에서 한번에 data_onehot_enco에 값입력

In [33]:

# le = LabelEncoder()
# oe = OneHotEncoder()

# le.fit(data['도로형태2'])
# road2 = le.transform(data['도로형태2'])
# road2 = road2.reshape(-1, 1)

# oe.fit(road2)
# oe_road2 = oe.transform(road2)


# oe_road2 = pd.DataFrame(oe_road2)


# data_label_enco = pd.merge(data_label_enco, oe_road2, left_index=True, right_index=True)

# data_label_enco = one_hot.drop(columns='도로형태2')
# data_label_enco = one_hot.rename(columns= {0 : '도로형태2'})

# data_label_enco.info()

In [34]:
# one_hot = pd.get_dummies(data, columns=['도로형태1', '도로형태2'])
# one_hot

# **------------------------------------------------------**

# **3. 구,동 인코딩**

In [35]:
# data = pd.concat([train_df, test_df], sort=False)

## **원-핫 인코딩 적용**

### 구
- `ohe_gu` : 원-핫 인코딩 적용된 배열
- `ohe_gu_df` : `ohe_gu` 의 df

**1. 구 정보 확인**

In [36]:
label_gu = data['구']
print(label_gu.unique())
print(label_gu.shape)
data.info()

['중구' '달서구' '수성구' '북구' '동구' '서구' '달성군' '남구']
(50572,)
<class 'pandas.core.frame.DataFrame'>
Index: 50572 entries, 0 to 10962
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      50572 non-null  object
 1   요일      50572 non-null  object
 2   기상상태    50572 non-null  object
 3   노면상태    50572 non-null  object
 4   사고유형    50572 non-null  object
 5   연       50572 non-null  int64 
 6   월       50572 non-null  int64 
 7   일       50572 non-null  int64 
 8   시간      50572 non-null  int64 
 9   도시      50572 non-null  object
 10  구       50572 non-null  object
 11  동       50572 non-null  object
 12  도로형태1   50572 non-null  object
 13  도로형태2   50572 non-null  object
dtypes: int64(4), object(10)
memory usage: 5.8+ MB


**2. 원-핫 인코딩 적용**

In [37]:
# 1. OneHotEncoder 임포트 및 정의
ohe = OneHotEncoder(sparse=False) # array 반환을 위해 sparse=False

# 2. 2차원 데이터로 반환
label_gu_rs = label_gu.values.reshape(-1, 1)

# 3. One-Hot Encoding 적용
ohe.fit(label_gu_rs)

# 4. One-Hot Encoding 적용된 배열 리턴
ohe_gu = ohe.transform(label_gu_rs)

In [38]:
print('One-Hot Encoding 적용된 배열: ', ohe_gu)
print('One-Hot Encoding 카테고리 확인: ', ohe.categories_)

One-Hot Encoding 적용된 배열:  [[0. 0. 0. ... 0. 0. 1.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
One-Hot Encoding 카테고리 확인:  [array(['남구', '달서구', '달성군', '동구', '북구', '서구', '수성구', '중구'], dtype=object)]


In [39]:
ohe_gu_df = pd.DataFrame(ohe_gu, columns=ohe.categories_[0])
ohe_gu_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50572 entries, 0 to 50571
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   남구      50572 non-null  float64
 1   달서구     50572 non-null  float64
 2   달성군     50572 non-null  float64
 3   동구      50572 non-null  float64
 4   북구      50572 non-null  float64
 5   서구      50572 non-null  float64
 6   수성구     50572 non-null  float64
 7   중구      50572 non-null  float64
dtypes: float64(8)
memory usage: 3.1 MB


## merge할때 새로 적은 코드-구 데이터 원핫인코딩-

In [40]:
data_onehot_enco = pd.get_dummies(data_onehot_enco, columns=['구'])
data_onehot_enco.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50572 entries, 0 to 10962
Data columns (total 41 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ID               50572 non-null  object
 1   요일               50572 non-null  object
 2   기상상태             50572 non-null  object
 3   연                50572 non-null  int64 
 4   월                50572 non-null  int64 
 5   일                50572 non-null  int64 
 6   시간               50572 non-null  int64 
 7   도시               50572 non-null  object
 8   동                50572 non-null  object
 9   사고유형_차대사람        50572 non-null  bool  
 10  사고유형_차대차         50572 non-null  bool  
 11  사고유형_차량단독        50572 non-null  bool  
 12  노면상태_건조          50572 non-null  bool  
 13  노면상태_기타          50572 non-null  bool  
 14  노면상태_서리/결빙       50572 non-null  bool  
 15  노면상태_적설          50572 non-null  bool  
 16  노면상태_젖음/습기       50572 non-null  bool  
 17  노면상태_침수          50572 non-null  boo

### 동
- `ohe_dong` : 원-핫 인코딩 적용된 배열
- `ohe_dong_df` : ohe_dong 의 df

**1. 동 정보 확인**

In [41]:
label_dong = data['동']
print(label_dong.unique())
print(label_dong.shape)

['대신동' '감삼동' '두산동' '복현동' '신암동' '지산동' '상인동' '태전동' '지묘동' '평리동' '신기동' '상리동'
 '월성동' '황금동' '구암동' '신천동' '만촌동' '남산동' '비산동' '침산동' '두류동' '신매동' '대곡동' '유가읍'
 '논공읍' '율하동' '시지동' '봉무동' '다사읍' '동산동' '이현동' '이천동' '수성동4가' '노원동3가' '서변동'
 '관음동' '학정동' '파동' '상동' '이곡동' '효목동' '읍내동' '남성로' '죽전동' '대명동' '동천동' '지저동'
 '사수동' '덕산동' '본리동' '용계동' '범어동' '신당동' '성당동' '용산동' '태평로3가' '도학동' '욱수동' '불로동'
 '칠성동1가' '장기동' '매호동' '대천동' '삼덕동1가' '수성동2가' '팔달동' '삼덕동' '진천동' '수성동3가' '가창면'
 '방촌동' '산격동' '월암동' '현풍읍' '구지면' '매천동' '송현동' '고성동3가' '봉덕동' '칠성동2가' '공평동'
 '봉산동' '괴전동' '국우동' '대현동' '노원동1가' '대봉동' '본동' '신서동' '검단동' '화원읍' '동호동'
 '삼덕동2가' '입석동' '중동' '삼덕동3가' '파호동' '동내동' '하빈면' '율암동' '동인동1가' '내당동' '유천동'
 '호산동' '교동' '옥포읍' '원대동3가' '검사동' '동인동4가' '수성동1가' '고성동2가' '각산동' '중리동'
 '고성동1가' '갈산동' '대림동' '호림동' '연호동' '동변동' '범물동' '계산동2가' '동문동' '고모동' '금호동'
 '장동' '도원동' '수창동' '동인동2가' '가천동' '노변동' '달성동' '동인동3가' '서문로2가' '동성로2가' '중대동'
 '사월동' '장관동' '신평동' '숙천동' '성동' '동일동' '시장북로' '서호동' '용수동' '대흥동' '동성로3가' '연경동'
 '원대동1가' '남일동' '부동' '송정동' '백안동' '도동' '인교동' '미대동' '노원동2가' '북성로1가

**2. 원-핫 인코딩 적용**

In [42]:
# # 1. OneHotEncoder 임포트 및 정의
# ohe = OneHotEncoder(sparse=False) # array 반환을 위해 sparse=False

# # 2. 2차원 데이터로 반환
# label_dong_rs = label_dong.values.reshape(-1, 1)

# # 3. One-Hot Encoding 적용
# ohe.fit(label_dong_rs)

# # 4. One-Hot Encoding 적용된 배열 리턴
# ohe_dong = ohe.transform(label_dong_rs)

In [43]:
# print('One-Hot Encoding 적용된 배열: ', ohe_dong)
# print('One-Hot Encoding 카테고리 확인: ', ohe.categories_)

In [44]:
# ohe_dong_df = pd.DataFrame(ohe_dong, columns=ohe.categories_[0])
# ohe_dong_df

## merge할때 새로 적은 코드 - 동 데이터 원핫인코딩-

In [45]:
# data_onehot_enco = pd.get_dummies(data_onehot_enco, columns=['동'])
# data_onehot_enco.info()

## 구,동 레이블 인코딩 적용

### 구
- `le_gu` : 레이블 인코딩 적용된 배열
- `le_gu_df` : `le_gu` 의 df

In [46]:
from sklearn.preprocessing import LabelEncoder

**1. 구 정보 확인**

In [47]:
label_gu = data['구']
print(label_gu.unique())
print(label_gu.shape)

['중구' '달서구' '수성구' '북구' '동구' '서구' '달성군' '남구']
(50572,)


**2. 레이블 인코딩 적용**

In [48]:
# 1. LabelEncoder 임포트 및 정의
le = LabelEncoder()

# 2. 2차원 데이터로 반환
label_gu_rs = label_gu.values.reshape(-1, 1)

# 2. LabelEncoder 적용
le_gu = le.fit_transform(label_gu_rs)

### 라벨인코딩 기존 코드에서 작업하기위해 추가한 코드

In [49]:
label_encoder_gu = LabelEncoder()

# "사고유형" 컬럼 Label Encoding
data_label_enco['구'] = label_encoder_gu.fit_transform(data['구'])

# "노면상태" 컬럼 Label Encoding
data_label_enco['구'] = label_encoder_gu.fit_transform(data['구'])

# 결과 확인
data_label_enco.info()


<class 'pandas.core.frame.DataFrame'>
Index: 50572 entries, 0 to 10962
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      50572 non-null  object
 1   요일      50572 non-null  object
 2   기상상태    50572 non-null  object
 3   노면상태    50572 non-null  int32 
 4   사고유형    50572 non-null  int32 
 5   연       50572 non-null  int64 
 6   월       50572 non-null  int64 
 7   일       50572 non-null  int64 
 8   시간      50572 non-null  int64 
 9   도시      50572 non-null  object
 10  구       50572 non-null  int32 
 11  동       50572 non-null  object
 12  도로형태1   50572 non-null  int32 
 13  도로형태2   50572 non-null  int32 
dtypes: int32(5), int64(4), object(5)
memory usage: 4.8+ MB


In [50]:
print(set(le_gu))
print(le.classes_)
print(le.inverse_transform([0, 1, 2, 3, 4, 5, 6, 7]))

{0, 1, 2, 3, 4, 5, 6, 7}
['남구' '달서구' '달성군' '동구' '북구' '서구' '수성구' '중구']
['남구' '달서구' '달성군' '동구' '북구' '서구' '수성구' '중구']


In [51]:
le_gu_df = pd.DataFrame(le_gu, columns=['구'])
le_gu_df

,구
0,7
1,1
2,6
3,4
4,3
...,...
50567,0
50568,6
50569,6
50570,6


### 동
- `le_dong` : 레이블 인코딩 적용된 배열
- `le_dong_df` : le_dong 의 df

**1. 동 정보 확인**

In [52]:
label_dong = data['동']
print(label_dong.unique())
print(label_dong.shape)

['대신동' '감삼동' '두산동' '복현동' '신암동' '지산동' '상인동' '태전동' '지묘동' '평리동' '신기동' '상리동'
 '월성동' '황금동' '구암동' '신천동' '만촌동' '남산동' '비산동' '침산동' '두류동' '신매동' '대곡동' '유가읍'
 '논공읍' '율하동' '시지동' '봉무동' '다사읍' '동산동' '이현동' '이천동' '수성동4가' '노원동3가' '서변동'
 '관음동' '학정동' '파동' '상동' '이곡동' '효목동' '읍내동' '남성로' '죽전동' '대명동' '동천동' '지저동'
 '사수동' '덕산동' '본리동' '용계동' '범어동' '신당동' '성당동' '용산동' '태평로3가' '도학동' '욱수동' '불로동'
 '칠성동1가' '장기동' '매호동' '대천동' '삼덕동1가' '수성동2가' '팔달동' '삼덕동' '진천동' '수성동3가' '가창면'
 '방촌동' '산격동' '월암동' '현풍읍' '구지면' '매천동' '송현동' '고성동3가' '봉덕동' '칠성동2가' '공평동'
 '봉산동' '괴전동' '국우동' '대현동' '노원동1가' '대봉동' '본동' '신서동' '검단동' '화원읍' '동호동'
 '삼덕동2가' '입석동' '중동' '삼덕동3가' '파호동' '동내동' '하빈면' '율암동' '동인동1가' '내당동' '유천동'
 '호산동' '교동' '옥포읍' '원대동3가' '검사동' '동인동4가' '수성동1가' '고성동2가' '각산동' '중리동'
 '고성동1가' '갈산동' '대림동' '호림동' '연호동' '동변동' '범물동' '계산동2가' '동문동' '고모동' '금호동'
 '장동' '도원동' '수창동' '동인동2가' '가천동' '노변동' '달성동' '동인동3가' '서문로2가' '동성로2가' '중대동'
 '사월동' '장관동' '신평동' '숙천동' '성동' '동일동' '시장북로' '서호동' '용수동' '대흥동' '동성로3가' '연경동'
 '원대동1가' '남일동' '부동' '송정동' '백안동' '도동' '인교동' '미대동' '노원동2가' '북성로1가

**2. 레이블 인코딩 적용**

### 라벨인코딩 기존 코드에서 작업하기위해 추가한 코드

In [53]:
label_encoder_gu = LabelEncoder()

# "사고유형" 컬럼 Label Encoding
data_label_enco['동'] = label_encoder_gu.fit_transform(data['동'])

# "노면상태" 컬럼 Label Encoding
data_label_enco['동'] = label_encoder_gu.fit_transform(data['동'])

# 결과 확인
data_label_enco.info()


<class 'pandas.core.frame.DataFrame'>
Index: 50572 entries, 0 to 10962
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      50572 non-null  object
 1   요일      50572 non-null  object
 2   기상상태    50572 non-null  object
 3   노면상태    50572 non-null  int32 
 4   사고유형    50572 non-null  int32 
 5   연       50572 non-null  int64 
 6   월       50572 non-null  int64 
 7   일       50572 non-null  int64 
 8   시간      50572 non-null  int64 
 9   도시      50572 non-null  object
 10  구       50572 non-null  int32 
 11  동       50572 non-null  int32 
 12  도로형태1   50572 non-null  int32 
 13  도로형태2   50572 non-null  int32 
dtypes: int32(6), int64(4), object(4)
memory usage: 4.6+ MB


In [54]:
# 1. LabelEncoder 임포트 및 정의
le = LabelEncoder()

# 2. 2차원 데이터로 반환
label_dong_rs = label_dong.values.reshape(-1, 1)

# 2. LabelEncoder 적용
le_dong = le.fit_transform(label_dong_rs)
print(set(le_dong))
print(le.classes_)

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195}
['가창면' '가천동' '각산동' '갈산동' '감삼동' '검단동' '검사동' '계산동1가' '계산동2가' '고모동' '고성동1가'
 '고성동2가' '고성동3가' '공평동' '관음동' '괴전동' '교동' '구암동' '구지면' '국우동

In [55]:
le_dong_df = pd.DataFrame(le_dong, columns=['동'])
le_dong_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50572 entries, 0 to 50571
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   동       50572 non-null  int32
dtypes: int32(1)
memory usage: 197.7 KB


# **------------------------------------------------------**

# **4. 연,월,시 원핫인코딩**


#### 연,월,(일), 시간 최빈값 알아보기

In [56]:
#최빈값 알아보기
data['연'].value_counts().sort_index()

연
2019    14536
2020    12940
2021    12133
2022    10963
Name: count, dtype: int64

In [57]:
train_df['월'].value_counts()

월
10    3666
11    3577
5     3474
9     3413
6     3407
7     3381
8     3300
12    3262
4     3251
3     3052
1     3030
2     2796
Name: count, dtype: int64

In [58]:
data['월'].value_counts()

월
10    4638
11    4599
5     4514
9     4366
7     4364
6     4339
4     4186
8     4168
12    4142
1     3904
3     3848
2     3504
Name: count, dtype: int64

In [59]:
data['일'].value_counts()

일
11    1736
21    1728
8     1724
14    1717
27    1705
9     1700
4     1688
7     1685
18    1677
28    1676
10    1671
22    1671
15    1666
23    1665
20    1662
26    1657
24    1649
29    1647
17    1644
6     1639
25    1637
13    1635
16    1632
3     1631
12    1629
5     1618
2     1617
19    1569
30    1544
1     1544
31     909
Name: count, dtype: int64

### Chatpgt의 대답

Label Encoding: 순서나 순위가 있는 경우에 사용됩니다. 하지만, 년도는 연도 간의 순서를 나타내는 정량적인 정보보다는 범주형 정보로 인식하는 것이 더 적절합니다. Label Encoding은 연도 간의 관계를 숫자로 나타내기 때문에 모델이 연도 사이의 순서를 잘못 이해할 수 있습니다.

One-Hot Encoding: 각 연도를 별도의 피처로 인식하여 2019, 2020, 2021, 2022와 같이 각 연도를 0 또는 1로 표현하는 것입니다. 연도 간의 순서를 고려하지 않고 각각 독립된 피처로 다루기 때문에 모델이 연도 사이의 관계를 잘 파악할 수 있습니다. 또한, 연도 간의 교통사고 발생량의 변화를 모델이 명확히 이해할 수 있습니다.

주어진 상황에서는 교통사고 발생량의 추세와 관련이 있으므로, One-Hot Encoding을 사용하여 각 연도를 독립된 피처로 다루는 것이 더 적절할 것 같아요. 이렇게 하면 모델이 각 연도의 교통사고 발생량에 대한 패턴을 잘 파악할 수 있어요.

In [60]:
# 연도의 one-hot encoding 및 확인
data_onehot_enco = pd.get_dummies(data_onehot_enco, columns=['연'])

In [61]:
data_onehot_enco.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50572 entries, 0 to 10962
Data columns (total 44 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ID               50572 non-null  object
 1   요일               50572 non-null  object
 2   기상상태             50572 non-null  object
 3   월                50572 non-null  int64 
 4   일                50572 non-null  int64 
 5   시간               50572 non-null  int64 
 6   도시               50572 non-null  object
 7   동                50572 non-null  object
 8   사고유형_차대사람        50572 non-null  bool  
 9   사고유형_차대차         50572 non-null  bool  
 10  사고유형_차량단독        50572 non-null  bool  
 11  노면상태_건조          50572 non-null  bool  
 12  노면상태_기타          50572 non-null  bool  
 13  노면상태_서리/결빙       50572 non-null  bool  
 14  노면상태_적설          50572 non-null  bool  
 15  노면상태_젖음/습기       50572 non-null  bool  
 16  노면상태_침수          50572 non-null  bool  
 17  도로형태1_교차로        50572 non-null  boo

In [62]:
# 월의 one-hot encoding 및 확인
data_onehot_enco = pd.get_dummies(data_onehot_enco, columns=['월'])
data_onehot_enco.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50572 entries, 0 to 10962
Data columns (total 55 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ID               50572 non-null  object
 1   요일               50572 non-null  object
 2   기상상태             50572 non-null  object
 3   일                50572 non-null  int64 
 4   시간               50572 non-null  int64 
 5   도시               50572 non-null  object
 6   동                50572 non-null  object
 7   사고유형_차대사람        50572 non-null  bool  
 8   사고유형_차대차         50572 non-null  bool  
 9   사고유형_차량단독        50572 non-null  bool  
 10  노면상태_건조          50572 non-null  bool  
 11  노면상태_기타          50572 non-null  bool  
 12  노면상태_서리/결빙       50572 non-null  bool  
 13  노면상태_적설          50572 non-null  bool  
 14  노면상태_젖음/습기       50572 non-null  bool  
 15  노면상태_침수          50572 non-null  bool  
 16  도로형태1_교차로        50572 non-null  bool  
 17  도로형태1_기타         50572 non-null  boo

In [63]:
# 시간의 one-hot encoding 및 확인
data_onehot_enco = pd.get_dummies(data_onehot_enco, columns=['시간'])
data_onehot_enco.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50572 entries, 0 to 10962
Data columns (total 78 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ID               50572 non-null  object
 1   요일               50572 non-null  object
 2   기상상태             50572 non-null  object
 3   일                50572 non-null  int64 
 4   도시               50572 non-null  object
 5   동                50572 non-null  object
 6   사고유형_차대사람        50572 non-null  bool  
 7   사고유형_차대차         50572 non-null  bool  
 8   사고유형_차량단독        50572 non-null  bool  
 9   노면상태_건조          50572 non-null  bool  
 10  노면상태_기타          50572 non-null  bool  
 11  노면상태_서리/결빙       50572 non-null  bool  
 12  노면상태_적설          50572 non-null  bool  
 13  노면상태_젖음/습기       50572 non-null  bool  
 14  노면상태_침수          50572 non-null  bool  
 15  도로형태1_교차로        50572 non-null  bool  
 16  도로형태1_기타         50572 non-null  bool  
 17  도로형태1_단일로        50572 non-null  boo

이미 연도, 월, 시간은 숫자형 데이터로 되어 있기 때문에 Label Encoding을 하는 것은 좋지 않은 선택일 수 있습니다. Label Encoding은 주로 문자열 형태의 범주형 데이터를 숫자로 변환할 때 사용됩니다. 숫자형 데이터에 Label Encoding을 적용하면 모델이 해당 데이터를 범주형이라고 오해할 수 있고, 연속된 숫자 값의 크기를 고려할 수 있어 잘못된 학습을 유발할 수 있습니다.

따라서 이미 숫자로 표현된 연도, 월, 시간 데이터에 대해서는 Label Encoding을 적용할 필요가 없습니다. 이러한 경우에는 따로 전처리를 하지 않고 그대로 사용하는 것이 모델에 더 적합할 수 있습니다. 만약 이러한 데이터를 카테고리형 변수로 사용하고자 한다면, 해당 컬럼들을 One-Hot Encoding 등 다른 방법으로 처리하는 것이 좋을 수 있어요.

# **------------------------------------------------------**

# **5. 요일, 기상 엔코딩**

## Case 1: '요일' 컬럼 LabelEncoding
**0 = 금요일 / 1 = 목요일 / 2 = 수요일 / 3 = 월요일 / 4 = 일요일 / 5 = 토요일 / 6 = 화요일**로 변환됨

인코딩된 값 삽입위해 기존 '요일'컬럼 삭제

In [64]:
data_label_enco = data_label_enco.drop(data_label_enco.columns[1], axis=1).copy()

**요일 컬럼 LabelEncoding 후 컬럼 순서 다시 정렬**

In [65]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(data['요일'])
data_label_enco['요일'] = encoder.transform(data['요일'])
data_label_enco.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50572 entries, 0 to 10962
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      50572 non-null  object
 1   기상상태    50572 non-null  object
 2   노면상태    50572 non-null  int32 
 3   사고유형    50572 non-null  int32 
 4   연       50572 non-null  int64 
 5   월       50572 non-null  int64 
 6   일       50572 non-null  int64 
 7   시간      50572 non-null  int64 
 8   도시      50572 non-null  object
 9   구       50572 non-null  int32 
 10  동       50572 non-null  int32 
 11  도로형태1   50572 non-null  int32 
 12  도로형태2   50572 non-null  int32 
 13  요일      50572 non-null  int32 
dtypes: int32(7), int64(4), object(3)
memory usage: 4.4+ MB


**가독성 위해 컬럼 순서 원래 순서와 맞춰 정렬**

In [66]:
data_label_enco = data_label_enco[['ID', '요일', '기상상태', '노면상태', '사고유형', '연', '월', '일', '시간', '도시', '구', '동', '도로형태1', '도로형태2']]
display(data_label_enco.head())
print('인코딩 변환값 :', set(data_label_enco['요일']))

,ID,요일,기상상태,노면상태,사고유형,연,월,일,시간,도시,구,동,도로형태1,도로형태2
0,ACCIDENT_00000,6,맑음,0,0,2019,1,1,0,대구광역시,7,40,2,5
1,ACCIDENT_00001,6,흐림,0,0,2019,1,1,0,대구광역시,1,4,2,5
2,ACCIDENT_00002,6,맑음,0,0,2019,1,1,1,대구광역시,6,66,2,5
3,ACCIDENT_00003,6,맑음,0,1,2019,1,1,2,대구광역시,4,79,2,5
4,ACCIDENT_00004,6,맑음,0,1,2019,1,1,4,대구광역시,3,129,2,5


인코딩 변환값 : {0, 1, 2, 3, 4, 5, 6}


## Case 2 : 요일 컬럼 One-hot 인코딩 진행하기
- LabelEncoding 진행 시, 숫자값의 크고 작음이 있기에 몇몇 ML에서의 예측 성능이 떨어질 수 있음
- 그 이유로는 범주형 데이터를 수치형 데이터로만 변환을 하게 되면 **서로 간의 관계성**이 생기기 때문
- 예를 들어, 월요일을 1, 화요일을 2, 수요이을 3이라고 단순하게 수치형 데이터로 변환하게 되면 **해당 데이터들 간 1+2 = 3 이라는 관계성**이 존재하게 되는 문제가 있다
- **단, Tree 계열의 ML은 숫자의 이러한 특성을 반영하지 않으므로 괜찮다고 함**
- 위의 문제점을 방지하고자 요일 컬럼을 **one-hot 인코딩 방식인 get_dummies 방식**을 진행함

### **One-Hot 인코딩 결과**
해당 요일에 맞게 0과 1로 구성된 더미값들을 만듬

In [67]:
# data_onehot_enco.info()

In [68]:
data_onehot_enco = pd.get_dummies(data_onehot_enco, columns=['요일'])
data_onehot_enco.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50572 entries, 0 to 10962
Data columns (total 84 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ID               50572 non-null  object
 1   기상상태             50572 non-null  object
 2   일                50572 non-null  int64 
 3   도시               50572 non-null  object
 4   동                50572 non-null  object
 5   사고유형_차대사람        50572 non-null  bool  
 6   사고유형_차대차         50572 non-null  bool  
 7   사고유형_차량단독        50572 non-null  bool  
 8   노면상태_건조          50572 non-null  bool  
 9   노면상태_기타          50572 non-null  bool  
 10  노면상태_서리/결빙       50572 non-null  bool  
 11  노면상태_적설          50572 non-null  bool  
 12  노면상태_젖음/습기       50572 non-null  bool  
 13  노면상태_침수          50572 non-null  bool  
 14  도로형태1_교차로        50572 non-null  bool  
 15  도로형태1_기타         50572 non-null  bool  
 16  도로형태1_단일로        50572 non-null  bool  
 17  도로형태1_미분류        50572 non-null  boo

**인코딩한 label 값들을 기존 dataset과 병합**

In [69]:
# data_onehot_enco = data.drop(data.columns[1], axis=1).copy()

In [70]:
# data_onehot_enco = pd.concat([data_onehot_enco, data_onehot], axis=1)
# data_onehot_enco.info()

**가독성 위해 컬럼 순서 원래 순서와 맞춰 정렬** 

In [71]:
# data_onehot_enco = data_onehot_enco[['ID', '월요일', '화요일', '수요일', '목요일', '금요일', '토요일', '일요일', '기상상태', '노면상태',
#                                           '사고유형', '연', '월', '일', '시간', '도시', '구', '동', '도로형태1', '도로형태2']]
# data_onehot_enco.info()

In [72]:
# 요일 컬럼 순서 변경 (75번부터 80번 컬럼)
desired_order = ['요일_월요일', '요일_화요일', '요일_수요일', '요일_목요일', '요일_금요일', '요일_토요일', '요일_일요일']
data_onehot_enco = data_onehot_enco[ [col for col in data_onehot_enco.columns if col not in desired_order] + desired_order]


In [1]:
data_onehot_enco.info()

NameError: name 'data_onehot_enco' is not defined

## Case 3 : '기상상태' 컬럼 LabelEncoding 진행
**기상상태 컬럼에 대해 LabelEncoding 및 One-Hot 인코딩 실시**

In [74]:
encoder_weather = LabelEncoder()
encoder_weather.fit(data['기상상태'])
data_label_enco['기상상태'] = encoder_weather.transform(data['기상상태'])

display(data_label_enco)
print('인코딩 변환값 : ', set(data_label_enco['기상상태']))

,ID,요일,기상상태,노면상태,사고유형,연,월,일,시간,도시,구,동,도로형태1,도로형태2
0,ACCIDENT_00000,6,2,0,0,2019,1,1,0,대구광역시,7,40,2,5
1,ACCIDENT_00001,6,5,0,0,2019,1,1,0,대구광역시,1,4,2,5
2,ACCIDENT_00002,6,2,0,0,2019,1,1,1,대구광역시,6,66,2,5
3,ACCIDENT_00003,6,2,0,1,2019,1,1,2,대구광역시,4,79,2,5
4,ACCIDENT_00004,6,2,0,1,2019,1,1,4,대구광역시,3,129,2,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10958,ACCIDENT_50567,5,2,0,1,2022,12,31,18,대구광역시,0,38,2,9
10959,ACCIDENT_50568,5,2,0,1,2022,12,31,18,대구광역시,6,123,2,5
10960,ACCIDENT_50569,5,2,0,1,2022,12,31,20,대구광역시,6,134,2,5
10961,ACCIDENT_50570,5,2,0,1,2022,12,31,20,대구광역시,6,77,0,2


인코딩 변환값 :  {0, 1, 2, 3, 4, 5}


In [75]:
data_label_enco.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50572 entries, 0 to 10962
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      50572 non-null  object
 1   요일      50572 non-null  int32 
 2   기상상태    50572 non-null  int32 
 3   노면상태    50572 non-null  int32 
 4   사고유형    50572 non-null  int32 
 5   연       50572 non-null  int64 
 6   월       50572 non-null  int64 
 7   일       50572 non-null  int64 
 8   시간      50572 non-null  int64 
 9   도시      50572 non-null  object
 10  구       50572 non-null  int32 
 11  동       50572 non-null  int32 
 12  도로형태1   50572 non-null  int32 
 13  도로형태2   50572 non-null  int32 
dtypes: int32(8), int64(4), object(2)
memory usage: 4.2+ MB


인코딩 결과 : 0 - 기타 / 1 - 눈 / 2 - 맑음 / 3 - 비 / 4 - 안개 / 5 - 흐림

## Case 4 : '기상상태' 컬럼 OneHot 인코딩 진행

**get_dummies 함수 이용해 원핫 인코딩 진행**

In [76]:
data_onehot_enco = pd.get_dummies(data_onehot_enco, columns=['기상상태'])

In [77]:
data_onehot_enco.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50572 entries, 0 to 10962
Data columns (total 89 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ID               50572 non-null  object
 1   일                50572 non-null  int64 
 2   도시               50572 non-null  object
 3   동                50572 non-null  object
 4   사고유형_차대사람        50572 non-null  bool  
 5   사고유형_차대차         50572 non-null  bool  
 6   사고유형_차량단독        50572 non-null  bool  
 7   노면상태_건조          50572 non-null  bool  
 8   노면상태_기타          50572 non-null  bool  
 9   노면상태_서리/결빙       50572 non-null  bool  
 10  노면상태_적설          50572 non-null  bool  
 11  노면상태_젖음/습기       50572 non-null  bool  
 12  노면상태_침수          50572 non-null  bool  
 13  도로형태1_교차로        50572 non-null  bool  
 14  도로형태1_기타         50572 non-null  bool  
 15  도로형태1_단일로        50572 non-null  bool  
 16  도로형태1_미분류        50572 non-null  bool  
 17  도로형태1_주차장        50572 non-null  boo

In [78]:
# data_onehot_weather = pd.get_dummies(data['기상상태'], dtype=float)
# data_onehot_weather

In [79]:
# data_onehot_enco = data_onehot_enco.drop(data_onehot_enco.columns[8], axis=1)

In [80]:
# data_onehot_enco = pd.concat([data_onehot_enco, data_onehot_weather], axis=1)
# data_onehot_enco

In [81]:
# data_onehot_enco = data_onehot_enco[['ID', '월요일', '화요일', '수요일', '목요일', '금요일', '토요일', '일요일', '기타', '눈', '맑음', '비', '안개', '흐림', '노면상태',
#                                           '사고유형', '연', '월', '일', '시간', '도시', '구', '동', '도로형태1', '도로형태2']]
# data_onehot_enco

**인코딩된 컬럼들이 제대로 들어갔는지 확인**

In [82]:
data_onehot_enco.columns

Index(['ID', '일', '도시', '동', '사고유형_차대사람', '사고유형_차대차', '사고유형_차량단독', '노면상태_건조',
       '노면상태_기타', '노면상태_서리/결빙', '노면상태_적설', '노면상태_젖음/습기', '노면상태_침수',
       '도로형태1_교차로', '도로형태1_기타', '도로형태1_단일로', '도로형태1_미분류', '도로형태1_주차장',
       '도로형태2_고가도로위', '도로형태2_교량', '도로형태2_교차로부근', '도로형태2_교차로안',
       '도로형태2_교차로횡단보도내', '도로형태2_기타', '도로형태2_미분류', '도로형태2_주차장',
       '도로형태2_지하차도(도로)내', '도로형태2_터널', '구_남구', '구_달서구', '구_달성군', '구_동구', '구_북구',
       '구_서구', '구_수성구', '구_중구', '연_2019', '연_2020', '연_2021', '연_2022', '월_1',
       '월_2', '월_3', '월_4', '월_5', '월_6', '월_7', '월_8', '월_9', '월_10', '월_11',
       '월_12', '시간_0', '시간_1', '시간_2', '시간_3', '시간_4', '시간_5', '시간_6', '시간_7',
       '시간_8', '시간_9', '시간_10', '시간_11', '시간_12', '시간_13', '시간_14', '시간_15',
       '시간_16', '시간_17', '시간_18', '시간_19', '시간_20', '시간_21', '시간_22', '시간_23',
       '요일_월요일', '요일_화요일', '요일_수요일', '요일_목요일', '요일_금요일', '요일_토요일', '요일_일요일',
       '기상상태_기타', '기상상태_눈', '기상상태_맑음', '기상상태_비', '기상상태_안개', '기상상태_흐림'],
      dtype='object')

## 원핫 인코딩 label에 대한 다중공선성 확인

In [83]:
data_onehot = data_onehot_enco.drop(data_onehot_enco.columns[0], axis=1)

In [84]:
column_range = list(range(13))
data_corr = data_onehot.iloc[:, column_range]
data_corr

,일,도시,동,사고유형_차대사람,사고유형_차대차,사고유형_차량단독,노면상태_건조,노면상태_기타,노면상태_서리/결빙,노면상태_적설,노면상태_젖음/습기,노면상태_침수,도로형태1_교차로
0,1,대구광역시,대신동,True,False,False,True,False,False,False,False,False,False
1,1,대구광역시,감삼동,True,False,False,True,False,False,False,False,False,False
2,1,대구광역시,두산동,True,False,False,True,False,False,False,False,False,False
3,1,대구광역시,복현동,False,True,False,True,False,False,False,False,False,False
4,1,대구광역시,신암동,False,True,False,True,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10958,31,대구광역시,대명동,False,True,False,True,False,False,False,False,False,False
10959,31,대구광역시,시지동,False,True,False,True,False,False,False,False,False,False
10960,31,대구광역시,연호동,False,True,False,True,False,False,False,False,False,False
10961,31,대구광역시,범물동,False,True,False,True,False,False,False,False,False,True


**인코딩된 feature들 간의 상관계수 확인**

In [85]:
# data_corr.corr()

In [86]:
# import seaborn as sns
# sns.heatmap(data_corr.corr(), annot = True)

- feature간 상관계수를 통해 다중공선성 확인 결과, 뚜렷하게 나타나는 변수 간 강한 양의 상관관계는 없는 것으로 보아
- 다중공선성에 대한 문제는 없는 것으로 보임
- VIF(Variance Inflation Factors)를 통해 확인해보는 방법도 존재
- https://aliencoder.tistory.com/17 - VIF 설명

## **DataFrame 정리**
1. **data_label_enco** : LabelEncoding 데이터프레임
2. **data_onehot_enco** : One-Hot Encoding 데이터프레임


# 최종 데이터

In [87]:
data_label_enco.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50572 entries, 0 to 10962
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      50572 non-null  object
 1   요일      50572 non-null  int32 
 2   기상상태    50572 non-null  int32 
 3   노면상태    50572 non-null  int32 
 4   사고유형    50572 non-null  int32 
 5   연       50572 non-null  int64 
 6   월       50572 non-null  int64 
 7   일       50572 non-null  int64 
 8   시간      50572 non-null  int64 
 9   도시      50572 non-null  object
 10  구       50572 non-null  int32 
 11  동       50572 non-null  int32 
 12  도로형태1   50572 non-null  int32 
 13  도로형태2   50572 non-null  int32 
dtypes: int32(8), int64(4), object(2)
memory usage: 4.2+ MB


In [88]:
data_onehot_enco.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50572 entries, 0 to 10962
Data columns (total 89 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ID               50572 non-null  object
 1   일                50572 non-null  int64 
 2   도시               50572 non-null  object
 3   동                50572 non-null  object
 4   사고유형_차대사람        50572 non-null  bool  
 5   사고유형_차대차         50572 non-null  bool  
 6   사고유형_차량단독        50572 non-null  bool  
 7   노면상태_건조          50572 non-null  bool  
 8   노면상태_기타          50572 non-null  bool  
 9   노면상태_서리/결빙       50572 non-null  bool  
 10  노면상태_적설          50572 non-null  bool  
 11  노면상태_젖음/습기       50572 non-null  bool  
 12  노면상태_침수          50572 non-null  bool  
 13  도로형태1_교차로        50572 non-null  bool  
 14  도로형태1_기타         50572 non-null  bool  
 15  도로형태1_단일로        50572 non-null  bool  
 16  도로형태1_미분류        50572 non-null  bool  
 17  도로형태1_주차장        50572 non-null  boo

### 통합된 데이터 세트(data)를 다시 train과 test 세트로 분리
- 합쳐둔 data 날짜에 맞춰서 다시 train_new, test_new로 분리하기

In [ ]:
train_new.info()

In [89]:
train_new  = data_onehot_enco[:len(train)]
test_new = data_onehot_enco[len(train):]

### 학습시킬 feature 선택하기

In [90]:
# 예측에 모든 컬럼 사용하는 법.
X_test = test_new.drop(columns=['ID','도시','동']).copy()
X_train = train_new[X_test.columns].copy()
y_train = train['ECLO'].copy()

# 1. Logistic Regression

In [91]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
lr = LogisticRegression()
# 평가지표를 위해 train데이터에서 타겟 나누기
data_merged = pd.concat([X_train, y_train], axis=1)
X = data_merged.drop('ECLO', axis=1)
y = data_merged['ECLO']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [92]:
# 모델 훈련
lr.fit(X_train, y_train)

LogisticRegression()

In [93]:
# X_test예측
prediction_lr = lr.predict(X_test)
print(prediction_lr[0])

3


In [94]:

# 평가 지표 계산
# 결과 출력
print('정확도(train):{:.3f}'.format(accuracy_score(prediction_lr,y_test))) 

정확도(train):0.473


# 2. 서포트 벡터 머신(SVM)

In [95]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# SVM 모델 초기화
svm = SVC(kernel='linear')  # 선형 커널을 사용하려면 kernel='linear'로 설정합니다.


In [96]:
# SVM 모델 훈련
svm.fit(X_train, y_train)

KeyboardInterrupt: 

In [ ]:
# 검증 데이터에 대한 예측
prediction_svm = svm.predict(X_test)

# 정확도 평가
print('정확도(train):{:.3f}'.format(accuracy_score(prediction_svm,y_test))) 

# 3. LinearRegression

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
import numpy as np

# 선형 회귀 모델 초기화 및 학습
model_Lir = LinearRegression()
model_Lir.fit(X_train, y_train)

# 테스트 세트로 예측
prediction_Lir = model_Lir.predict(X_test)

# 모델 평가
mse = mean_squared_error(y_test, prediction_Lir)
r2 = r2_score(y_test, prediction_Lir)

print("MSE:", mse)
print("R-squared:", r2)

# # 학습된 모델의 계수 확인
# coefficients = pd.DataFrame({'Feature': X.columns, 'Coefficient': prediction_Lir.coef_})
# print(coefficients)

# 예측값과 실제값의 로그를 취하고, NaN을 0으로 대체
log_y_pred = np.log1p(prediction_Lir)
log_y_true = np.log1p(y_test)

# 로그를 취한 예측값과 실제값의 차이 계산
log_diff = log_y_pred - log_y_true

# 제곱 계산
log_diff_squared = np.square(log_diff)

# 평균 제곱근 오차(RMSE) 계산
rmsle = np.sqrt(np.mean(log_diff_squared))

print("RMSLE:", rmsle)

### rmlse줄이기-특성 스케일링

#### 크게 의미 없었음

In [ ]:
# 정규화, 다항회귀, 특성 스케일링 모두 의미X
# from sklearn.linear_model import Lasso, Ridge

# # L1 정규화 (Lasso)
# lasso = Lasso(alpha=0.1)
# lasso.fit(X_train_scaled, y_train)

# # L2 정규화 (Ridge)
# ridge = Ridge(alpha=0.1)
# ridge.fit(X_train_scaled, y_train)
# # 테스트 세트로 예측
# prediction_Lir = model_Lir.predict(X_test)

# # 모델 평가
# mse = mean_squared_error(y_test, prediction_Lir)
# r2 = r2_score(y_test, prediction_Lir)

# print("MSE:", mse)
# print("R-squared:", r2)

# # # 학습된 모델의 계수 확인
# # coefficients = pd.DataFrame({'Feature': X.columns, 'Coefficient': prediction_Lir.coef_})
# # print(coefficients)

# # 예측값과 실제값의 로그를 취하고, NaN을 0으로 대체
# log_y_pred = np.log1p(prediction_Lir)
# log_y_true = np.log1p(y_test)

# # 로그를 취한 예측값과 실제값의 차이 계산
# log_diff = log_y_pred - log_y_true

# # 제곱 계산
# log_diff_squared = np.square(log_diff)

# # 평균 제곱근 오차(RMSE) 계산
# rmsle = np.sqrt(np.mean(log_diff_squared))

# print("RMSLE:", rmsle)


### 이상치 삭제 해보기 

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
import numpy as np
from scipy import stats

def remove_outliers(df, threshold=3, features=None):
    if features is None:
        features = df.columns
    
    # 이상치를 제외한 행 선택
    mask = np.ones(len(df), dtype=bool)
    for feature in features:
        z_scores = np.abs(stats.zscore(df[feature]))
        outliers = np.where(z_scores > threshold)[0]
        mask[outliers] = False
    
    return df[mask]
# 사용 예시
# 노면상태_침수 도로형태2_터널
features_to_exclude_outliers = ['도로형태2_터널']
X_train_no_outliers = remove_outliers(X_train, threshold=3, features=features_to_exclude_outliers)
y_train_no_outliers = y_train.loc[X_train_no_outliers.index]

# 이상치 제거된 갯수 확인
num_removed_outliers = len(X_train) - len(X_train_no_outliers)
print(f'제거된 이상치 갯수: {num_removed_outliers}')

# 선형 회귀 모델 초기화 및 학습
model_Lir = LinearRegression()
model_Lir.fit(X_train_no_outliers, y_train_no_outliers)
prediction_Lir = model_Lir.predict(X_test)
# 예측값과 실제값의 로그를 취하고, NaN을 0으로 대체
log_y_pred = np.log1p(prediction_Lir)
log_y_true = np.log1p(y_test)
# 로그를 취한 예측값과 실제값의 차이 계산
log_diff = log_y_pred - log_y_true
# 제곱 계산
log_diff_squared = np.square(log_diff)
# 평균 제곱근 오차(RMSE) 계산
rmsle = np.sqrt(np.mean(log_diff_squared))
print("RMSLE:", rmsle)


#### 그냥몇개 삭제해봤는데 유의미하다. 그래서 아래에 정확히 영향을 끼치는 이상치를 찾아보았다. 가장 많은 이상치를 가진 컬럼은 월_10 이였다

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats

def find_and_show_outliers(df, column, threshold=3):
    z_scores = np.abs(stats.zscore(df[column]))
    outliers = np.where(z_scores > threshold)[0]
    num_outliers = len(outliers)
    
    # 이상치가 있는 샘플 출력
    if num_outliers > 0:
        print(f'{column}에서의 이상치 샘플:')
        print(df.iloc[outliers])
    else:
        print(f'{column}에서는 이상치가 없습니다.')

# 사용 예시
find_and_show_outliers(X_train, most_outliers_column, threshold=3)


### 이상치가 가장 많은 컬럼 3개까지 구해서 해당 컬럼에서 이상치를 제거해보았다. 

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats

def find_top_n_columns_with_most_outliers(df, n=3, threshold=3):
    columns_with_outliers = {}

    for column in df.columns:
        z_scores = np.abs(stats.zscore(df[column]))
        outliers = np.where(z_scores > threshold)[0]
        num_outliers = len(outliers)
        columns_with_outliers[column] = num_outliers
    
    # 이상치가 많은 순으로 정렬
    sorted_columns = sorted(columns_with_outliers.items(), key=lambda x: x[1], reverse=True)
    
    # 상위 n개 컬럼 추출
    top_n_columns = [column for column, _ in sorted_columns[:n]]
    
    return top_n_columns

# 사용 예시
top_3_columns_with_most_outliers = find_top_n_columns_with_most_outliers(X_train, n=3, threshold=3)

# 상위 3개 컬럼 출력
for i, column in enumerate(top_3_columns_with_most_outliers, start=1):
    print(top_3_columns_with_most_outliers)


## ['월_10', '월_11', '구_서구'] 이렇게 3개 뺐는데 rmsle가 왜 증가함????

## 이상치를 많이 빼도 문제. 컬럼 다 해본결과 도로형태2_터널의 이상치를 제거했을경우 가장 rmsle 낮음